In [1]:
import pandas as pd
import numpy as np
from Dora import Dora
import cython 
from numba import jit

data = pd.read_csv('weather_data_top100.csv')
data.head()

,date,tavg,tmin,tmax,prcp,snow,wspd,pres,country,city
0,1980-01-01,-3.0,-8.0,1.0,NaN,NaN,NaN,NaN,Afghanistan,Kabul
1,1980-01-02,-6.2,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,Kabul
2,1980-01-03,-7.7,-14.0,2.0,NaN,NaN,NaN,NaN,Afghanistan,Kabul
3,1980-01-04,-8.4,-15.0,-2.0,NaN,NaN,NaN,NaN,Afghanistan,Kabul
4,1980-01-05,-7.8,-14.0,NaN,NaN,NaN,NaN,NaN,Afghanistan,Kabul


In [2]:
dora_dataset = Dora(output='date', data=data.iloc[:, :-2])
dora_dataset.data

,date,tavg,tmin,tmax,prcp,snow,wspd,pres
0,1980-01-01,-3.0,-8.0,1.0,NaN,NaN,NaN,NaN
1,1980-01-02,-6.2,NaN,NaN,NaN,NaN,NaN,NaN
2,1980-01-03,-7.7,-14.0,2.0,NaN,NaN,NaN,NaN
3,1980-01-04,-8.4,-15.0,-2.0,NaN,NaN,NaN,NaN
4,1980-01-05,-7.8,-14.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
851507,2025-08-28,19.3,15.1,23.9,NaN,NaN,12.0,NaN
851508,2025-08-29,20.2,15.2,25.2,NaN,NaN,14.2,NaN
851509,2025-08-30,20.2,15.4,25.4,NaN,NaN,16.4,NaN
851510,2025-08-31,20.8,13.9,27.3,NaN,NaN,15.3,NaN


In [3]:
dora_dataset.impute_missing_values()
data_cleaned = dora_dataset.data

In [4]:
data_cleaned['country'] = data['country']
data_cleaned['city'] = data['city']
del data
data_cleaned

,date,tavg,tmin,tmax,prcp,snow,wspd,pres,country,city
0,1980-01-01,-3.0,-8.000000,1.000000,2.874872,5.813453,12.072765,1014.237086,Afghanistan,Kabul
1,1980-01-02,-6.2,13.502184,23.347088,2.874872,5.813453,12.072765,1014.237086,Afghanistan,Kabul
2,1980-01-03,-7.7,-14.000000,2.000000,2.874872,5.813453,12.072765,1014.237086,Afghanistan,Kabul
3,1980-01-04,-8.4,-15.000000,-2.000000,2.874872,5.813453,12.072765,1014.237086,Afghanistan,Kabul
4,1980-01-05,-7.8,-14.000000,23.347088,2.874872,5.813453,12.072765,1014.237086,Afghanistan,Kabul
...,...,...,...,...,...,...,...,...,...,...
851507,2025-08-28,19.3,15.100000,23.900000,2.874872,5.813453,12.000000,1014.237086,Yemen,Sana'a
851508,2025-08-29,20.2,15.200000,25.200000,2.874872,5.813453,14.200000,1014.237086,Yemen,Sana'a
851509,2025-08-30,20.2,15.400000,25.400000,2.874872,5.813453,16.400000,1014.237086,Yemen,Sana'a
851510,2025-08-31,20.8,13.900000,27.300000,2.874872,5.813453,15.300000,1014.237086,Yemen,Sana'a


In [5]:
def calc_statistic(sample1, sample2, n1, n2):
    s1_mean = sample1.mean()
    s2_mean = sample2.mean()
    s1_var = sample1.var()
    s2_var = sample2.var()
    T = (abs(s1_mean - s2_mean)/(n1*s1_var + n2*s2_var)**(1/2))*(n1*n2*(n1+n2-2)/(n1 + n2))**(1/2)
    return T


%timeit calc_statistic(data_cleaned['tmin'], data_cleaned['tmax'], data_cleaned['tmin'].size, data_cleaned['tmax'].size)

46.8 ms ± 532 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%load_ext Cython

In [16]:
%%cython
from cpython cimport array
import array

Content of stdout:
_cython_magic_2a79e679141f9d815f1e8837da71309b0b8a12afb5abdc93330296ad191fc014.c
D:\python\include\pyconfig.h(59): fatal error C1083: Cannot open include file: 'io.h': No such file or directory

In [ ]:
%%cython
# сконвертувати пандасівський серіес в пайтонівський еррей
cdef double calc_statistic(array.array sample1, array.array sample2, int n1, int n2):
    s1_mean = sum(sample1)/n1
    s2_mean = sum(sample2)/n2
    s1_var = (sum((sample1 - s1_mean)**2))/n1
    s2_var = (sum((sample2 - s2_mean)**2))/n2
    T = (abs(s1_mean - s2_mean)/(n1*s1_var + n2*s2_var)**(1/2))*(n1*n2*(n1+n2-2)/(n1 + n2))**(1/2)
    return T



Content of stdout:
_cython_magic_c2d5661b71e93bcbd6e75802dc1ceeab9e74c1a3b1d431fcef62aaa59c6e8340.c
D:\python\include\pyconfig.h(59): fatal error C1083: Cannot open include file: 'io.h': No such file or directory

In [13]:

%timeit calc_statistic(data_cleaned['tmin'], data_cleaned['tmax'], data_cleaned['tmin'].size, data_cleaned['tmax'].size)

50.8 ms ± 2.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
